In [1]:
import json
import os
import re
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from scrapy.crawler import CrawlerProcess
from mantis_scrapper.mantis_scrapper.spiders.mantis_spider import MantisSpider
from bs4 import BeautifulSoup
import sys
import os

In [2]:
print(sys.getrecursionlimit())
sys.setrecursionlimit(150000)
print(sys.getrecursionlimit())

3000
150000


In [3]:
news_web_tag_info = {
    'fiercewireless': {
        'url': 'https://www.fiercewireless.com',
        'tags': [
            [('div', 'id', 'article-tags-row'), ('a', 'class', 'tag')],
            [('div', 'id', 'article-tags-bottom-row'), ('a', 'class', 'tag')]
        ],
        'date': [
            [('span', 'class', 'date')]
        ]
    },
    'macrumors': {
        'url': 'https://www.macrumors.com',
        'tags': [
            [('div', 'class', 'linkback'), ('a', '', '')]
        ],
        'date': [
            [('time', '', '')]
        ]
    },
    'macworld': {
        'url': 'https://www.macworld.com',
        'tags': [],
        'date': [
            [('span', 'class', 'posted-on')]
        ]
    },
    'digitimes': {
        'url': 'https://www.digitimes.com',
        'tags': [
            [('div', 'class', 'news-content-frame'), ('a', 'class', 'label grey')]
        ],
        'date': [
            [('span', 'class', 'date'), ('time', '', '')]
        ]
    },
    'lightreading': {
        'url': 'https://www.lightreading.com',
        'tags': [],
        'date': [
            [('span', 'class', 'date'), ('time', '', '')]
        ]
    }
    ,
    'eetimes': {
        'url': 'https://www.eetimes.com',
        'tags': [
            [('div', 'class', 'post-tags'), ('span', 'itemprop', 'keywords'), ('a', 'rel', 'tag')]
        ],
        'date': [
            [('span', 'class', 'articleHeader-date')]
        ]
    }
    ,
    'seekingalpha': {
        'url': 'https://seekingalpha.com',
        'tags': [],
        'date': [
            [('span', 'class', 'oT-Ov oT-ji ag-gn')]
        ]
    },
    'gsma': {
        'url': 'https://www.gsma.com',
        'tags': [],
        'date': [
            [('p', 'class', 'post-date')]
        ]
    },
    'wsj': {
        'url': 'https://www.wsj.com',
        'tags': [],
        'date': [
            [('time', '', '')]
        ]
    }
}

In [4]:
news_webs = [
    'https://www.fiercewireless.com',
    'https://www.bloomberg.com',
    'https://www.digitimes.com',
    'https://www.macworld.com',
    'https://www.macrumors.com'
]

news_webs = news_webs[-3:-2]

key_words = [
    "smartphone", "smartphone technology",
    "wearables", "smart watch", "smart glasses", "ar/vr headsets", "ar headsets", "vr headsets",
    "iphone", "android smartphone",
    "apple", "samsung", "xiaomi", "oppo", "vivo",
    "qualcomm", "nxp", "mediaTek", "cirrus logic", "skyworks", "qorvo", "broadcom", "sony", "murata",
    "wifi", "cellular", "5g", "6g", "uwb", "nfc",
    "apple pay", "android pay",
    "magsafe", "wireless charging",
    "power management", "battery management",
    "envelope tracker",
    "rf front end", "transceiver", "modem",
    "mmwave", "fr2", "fr3",
    "oled display", "uled", "ltpo", "display driver ic",
    "smartphone camera", "image sensing", "3d sensing", "camera",
    "smart audio",
    "haptics",
    "satellite to smartphone connectivity"
]

WINDOW_SIZE = "600,400"

chrome_options = Options()
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

In [5]:
all_urls = []
scrapped_links = []
visited_links = []

In [6]:
def link_parser(web_links, out_file_name, url_info):
    urls = []
    for count, url in enumerate(web_links):
        if url.endswith('/'):
            url = url[:-1]
        if url in scrapped_links or url in visited_links:
            if len(web_links) > 1:
                print("skipping already visited: ", url)
                continue
        print("loading....", url)
        driver = webdriver.Chrome(
            executable_path=r'C:\chromedriver.exe',
            chrome_options=chrome_options
        )
        driver.get(url)
        # with open("visited_urls.txt", 'a') as fb:
        #     fb.write(url + "\n")
        #     visited_links.append(url)
        html = driver.page_source

        time.sleep(10)

        driver.close()
        driver.quit()
        soup = BeautifulSoup(html, 'html.parser')
        date = 'n/a'
        gather_link = True
        if len(url.split('/')) > 4:
            tags_list = []
            key_word_match = False
            matched_keys = []
            if url_info['tags']:
                for way in url_info['tags']:
                    current_pos = soup
                    for i, indnt in enumerate(way):
                        if i == len(way) - 1:
                            try:
                                current_pos = current_pos.find_all(indnt[0], {indnt[1]: indnt[-1]})
                            except Exception as e:
                                print("tag error ====>", len(current_pos), e)
                                exc_type, exc_obj, exc_tb = sys.exc_info()
                                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                                print(exc_type, fname, exc_tb.tb_lineno)
                                current_pos = []
                                gather_link = False
                        else:
                            current_pos = current_pos.find_all(indnt[0], {indnt[1]: indnt[-1]})
                            if len(current_pos) > 0:
                                current_pos = current_pos[0]

                    for tag in current_pos:
                        if tag and tag not in tags_list:
                            tags_list.append(tag.text)
                            if tag.text.lower() in key_words:
                                key_word_match = True
                                matched_keys.append(tag.text)
            else:
                for tag in key_words:
                    if tag.replace(' ', '-') in url.lower():
                        key_word_match = True
                        matched_keys.append(tag)
                        tags_list.append(tag)
            print(tags_list, matched_keys)
            if key_word_match:
                for way in url_info['date']:
                    current_pos = soup
                    for i, indnt in enumerate(way):
                        if i == len(way) - 1:
                            try:
                                current_pos = current_pos.find_all(indnt[0], {indnt[1]: indnt[-1]})
                            except Exception as e:
                                print("date error ====>", len(current_pos), e)
                                exc_type, exc_obj, exc_tb = sys.exc_info()
                                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                                print(exc_type, fname, exc_tb.tb_lineno)
                                current_pos = []
                                gather_link = False
                        else:
                            current_pos = current_pos.find_all(indnt[0], {indnt[1]: indnt[-1]})
                            if len(current_pos) > 0:
                                current_pos = current_pos[0]

                if len(current_pos) > 0:
                    try:
                        date = current_pos[0].text
                    except Exception as e:
                        print("date error ====>", e)
                        exc_type, exc_obj, exc_tb = sys.exc_info()
                        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                        print(exc_type, fname, exc_tb.tb_lineno)
                        try:
                            date = current_pos[0].getText()
                        except Exception as e:
                            print("date error ====>", e)
                            exc_type, exc_obj, exc_tb = sys.exc_info()
                            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                            print(exc_type, fname, exc_tb.tb_lineno)
                            date = ''
                            gather_link = False
                    print(date)
                with open(out_file_name, 'a') as fa:
                    fa.write(f"{url} | {date} | {';'.join(matched_keys)} | {';'.join(tags_list)} \n")
                    scrapped_links.append(url)

        if gather_link:
            for link in soup.find_all('a'):
                link_ = link.get('href')
                if link_:
                    if not link_.startswith('https://www.'):
                        if not 'www.' in link_:
                            base_url = "https://" + url.split('//')[1].split('/')[0]
                            link_ = base_url + link_
                        else:
                            if link_.startswith("/"):
                                link_ = link_[1:]
                            link_ = "https://" + link_
                    website = "https://" + link_.split('//')[1].split('/')[0]
                    # print(website, link_)
                    if website not in news_webs:
                        continue
                    if link_ in all_urls or link_ in scrapped_links or link_ in visited_links:
                        continue

                    if "/comment.asp" in link_:
                        continue
                    urls.append(link_)
                    all_urls.append(link_)

    link_parser(urls, out_file_name, url_info)


In [ ]:
file_name_out = "matched_urls_all.txt"

if os.path.exists(file_name_out):
    with open(file_name_out, 'r') as fao:
        scrapped_links = [x.split('|')[0].strip() for x in fao.readlines()]
else:
    with open(file_name_out, 'w') as fao:
        fao.write('')
# if os.path.exists("visited_urls.txt"):
#     with open("visited_urls.txt", 'r') as fbo:
#         visited_links = [x.replace('\n', '').strip() for x in fbo.readlines()]
for web in news_webs:
    info = news_web_tag_info[web.split('.')[1]]
    link_parser([web], out_file_name=file_name_out, url_info=info)

loading.... https://www.digitimes.com


C:\Users\admin\AppData\Local\Temp\ipykernel_10740\3123406037.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(
C:\Users\admin\AppData\Local\Temp\ipykernel_10740\3123406037.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


loading.... https://www.digitimes.com/tw
loading.... https://www.digitimes.com/cn
loading.... https://www.digitimes.com
loading.... https://www.digitimes.com/register
loading.... https://www.digitimes.com/newregister/myaccount.asp?view=general
tag error ====> 0 ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?
<class 'AttributeError'> 3123406037.py 38
[] []
loading.... https://www.digitimes.com/newregister/join.asp
tag error ====> 0 ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?
<class 'AttributeError'> 3123406037.py 38
[] []
loading.... https://www.digitimes.com/tech
loading.... https://www.digitimes.com/topic/semiconductors
tag error ====> 0 ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single eleme

In [ ]:
with open('all_urls.txt', 'w') as f:
    f.write("\n".join(all_urls))